In [3]:
import torch
import numpy as np
import matplotlib.pyplot as plt
from torchvision import datasets, transforms
import torchvision
from tqdm.auto import tqdm

In [24]:
device = "cuda" if torch.cuda.is_available() else "cpu"

In [4]:
test_path = r"C:\Users\pavle\OneDrive\Desktop\Torch\DOGS & CATS\dogs-vs-cats\test_path"
train_path = r"C:\Users\pavle\OneDrive\Desktop\Torch\DOGS & CATS\dogs-vs-cats\train_path"
torch.cuda.empty_cache()
train_transform = transforms.Compose([
    transforms.Resize((256,256)),
    transforms.RandomHorizontalFlip(),
    transforms.RandomResizedCrop(256),
    transforms.ToTensor()
])

test_transform = transforms.Compose([
    transforms.Resize((256,256)),
    transforms.RandomHorizontalFlip(),
    transforms.RandomResizedCrop(256),
    transforms.ToTensor()
])

In [5]:
train_dataset = datasets.ImageFolder(train_path, transform =train_transform)
test_dataset = datasets.ImageFolder(test_path, transform =test_transform)

print(f" len of  train_dataset {len(train_dataset)}, len of test_dataset = {len(test_dataset)}")


 len of  train_dataset 25000, len of test_dataset = 12500


In [23]:
BS = 32
train_dataloader = torch.utils.data.DataLoader(dataset = train_dataset,
                                            batch_size = BS,
                                            shuffle  = True)
test_dataloader = torch.utils.data.DataLoader(dataset = test_dataset,
                                            batch_size= BS,
                                            shuffle = True)
len(train_dataloader), len(test_dataloader)


(782, 391)

In [7]:
from torch import nn
class CatDogClassification(nn.Module):
    def __init__(self,
                input_shape,
                hidden_units,
                output_shape):
        super().__init__()

        self.layer_block1 = nn.Sequential(
            nn.Conv2d(in_channels = input_shape,
                        out_channels = hidden_units,
                        kernel_size = 3,
                        stride = 1,
                        padding = 1),
            nn.ReLU(),
            nn.Conv2d(in_channels = hidden_units,
                        out_channels= hidden_units,
                        kernel_size = 3,
                        stride =1,
                        padding = 1),
            nn.ReLU(),
            nn.MaxPool2d(kernel_size = 2))

        self.layer_block2 = nn.Sequential(
            nn.Conv2d(in_channels = hidden_units,
                        out_channels = hidden_units,
                        kernel_size = 3,
                        stride = 1,
                        padding = 1),
            nn.ReLU(),
            nn.Conv2d(in_channels = hidden_units,
                        out_channels= hidden_units,
                        kernel_size = 3,
                        stride =1,
                        padding = 1),
            nn.ReLU(),
            nn.MaxPool2d(kernel_size = 2))
        self.classification_layer = nn.Sequential(
            nn.Flatten(),
            nn.Linear(in_features = hidden_units*4096,
            out_features= output_shape),
            nn.Sigmoid())

    def forward(self, x):
        x = self.layer_block1(x)
        x = self.layer_block2(x)
        x = self.classification_layer(x)
        return x
        
    


In [15]:
from torch.optim import optimizer

model  = CatDogClassification(3,16,1)
model.to(device)
loss_fn = torch.nn.BCEWithLogitsLoss()
optimizer = torch.optim.Adam(model.parameters(), lr = 0.001)

In [16]:
def train_step(model,
                dataloader,
                loss_fn, 
                optimizer):

    model.train()

    train_loss = 0
    train_acc = 0

    for batch, (X,y) in enumerate(dataloader):
        
        X, y = X.to(device), y.to(device)
        train_preds = model(X)
        
        loss = loss_fn(train_preds, y.unsqueeze(dim = 1 ).float())

        train_loss += loss
        pred = torch.argmax(train_preds, dim = 1)
        train_acc += (pred == y).float().mean()

        optimizer.zero_grad()
        loss.backward()
        optimizer.step()

    train_loss /= len(dataloader)
    train_acc /= len(dataloader)

    print(f"train_loss ={train_loss:2f}, train_acc = {train_acc:2f}")

def test_step(model,
            dataloader,
            loss_fn):
    
    model.eval()
    test_acc = 0
    test_loss = 0

    with torch.inference_mode():

        for X, y in dataloader:
            
            X, y = X.to(device), y.to(device)
            test_preds = model(X)

            loss = loss_fn(test_preds, y.unsqueeze(dim = 1).float())
            test_loss += loss

            pred = torch.argmax(test_preds, dim =1 )
            test_acc += (pred == y).float().mean()

        test_loss /= len(dataloader)
        test_acc /= len(dataloader)

    print(f"test_loss = {test_loss:2f}, test_acc = {test_acc:2f}")




In [ ]:
for i in tqdm(range(3)):

    train_step(model,
        train_dataloader,
        loss_fn,
        optimizer)

    test_step(model,
        test_dataloader,
        loss_fn)